## Import lib

In [18]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
import cv2
import os, sys
import geopandas as gpd
from zkyhaxpy import gis_tools, io_tools

## Define functions

## Load data

In [2]:
df_predict_pm25_monthly = pd.read_parquet(r'../data/df_predict_pm25_monthly_v2.parquet')

## Animated - Time Series

In [67]:
df_pm25_aod = df_predict_pm25_monthly.mean(axis=0).reset_index()
df_pm25_aod.columns = ['year_month', 'pm25']

df_pm25_aod['source'] = 'Model (MODIS-AOD)'
df_pm25_aod

,year_month,pm25,source
0,2000-02,52.192121,Model (MODIS-AOD)
1,2000-03,65.271983,Model (MODIS-AOD)
2,2000-04,73.412128,Model (MODIS-AOD)
3,2000-05,58.596678,Model (MODIS-AOD)
4,2000-06,53.313467,Model (MODIS-AOD)
...,...,...,...
285,2023-12,33.220837,Model (MODIS-AOD)
286,2024-01,26.772110,Model (MODIS-AOD)
287,2024-02,33.048157,Model (MODIS-AOD)
288,2024-03,67.537365,Model (MODIS-AOD)


## Load data from OpenAQ

In [13]:

path_df_openaq = r'../data/gdf_openaq_chiangmai_by_location.parquet'
if os.path.exists(path_df_openaq):
    df_openaq = pd.read_parquet(path_df_openaq)
    print(f'{path_df_openaq} has been loaded')
else:
    gdf_openaq = gpd.read_file('../data/gdf_openaq_chiangmai_by_location.gpkg')
    print('gdf_openaq has been loaded.')
    if gdf_openaq.index.name is None:
        gdf_openaq = gdf_openaq.set_index('measurement_id')
    
    df_openaq = gdf_openaq.drop(columns=['geometry']).copy()
    df_openaq.to_parquet(path_df_openaq)
    print(f'{path_df_openaq} has been saved')

../data/gdf_openaq_chiangmai_by_location.parquet has been loaded


In [19]:
if os.path.exists('../data/gdf_openaq_location_chiangmai_v2.gpkg'):
    gdf_openaq_location_chiangmai = gpd.read_file(r'../data/gdf_openaq_location_chiangmai_v2.gpkg')

else:
    df_openaq_location = df_openaq.copy()
    df_openaq_location = df_openaq_location.reindex(columns=['locationId', 'location', 'sensorType', 'entity', 'lat', 'lon']).drop_duplicates()

    df_openaq_location['geometry'] = 'POINT (' + df_openaq_location['lon'].astype(str) + ' ' + df_openaq_location['lat'].astype(str) + ')'

    gdf_openaq_location = gis_tools.df_to_gdf(df_openaq_location, 'geometry')


    gdf_province = gpd.read_file(r'..\data\thailandWithName.json')
    gdf_chiangmai = gdf_province[gdf_province['name'] == 'Chiang Mai']

    gdf_openaq_location_chiangmai = gdf_openaq_location[gdf_openaq_location.geometry.within(gdf_chiangmai.geometry.iloc[0])]
    gdf_openaq_location_chiangmai.to_file(r'../data/gdf_openaq_location_chiangmai_v2.gpkg')

In [64]:
df_openaq = df_openaq[df_openaq['locationId'].isin(gdf_openaq_location_chiangmai['locationId'].to_list())].copy()

#Remove some end of April'21 datapoint 
df_openaq = df_openaq[df_openaq['date']>='2021-05-01'].copy()

#Get year month column
df_openaq['year_month'] = df_openaq['date'].str.slice(0, 7)

#Calculate monthly mean
df_pm25_openaq = df_openaq.groupby('year_month', as_index=False).agg(pm25=('value', 'mean'))
df_pm25_openaq['source'] = 'OpenAQ'

In [ ]:
#S

# Combine 2 data sources

In [68]:
df_pm25 = pd.concat([df_pm25_aod, df_pm25_openaq])
df_pm25

,year_month,pm25,source
0,2000-02,52.192121,Model (MODIS-AOD)
1,2000-03,65.271983,Model (MODIS-AOD)
2,2000-04,73.412128,Model (MODIS-AOD)
3,2000-05,58.596678,Model (MODIS-AOD)
4,2000-06,53.313467,Model (MODIS-AOD)
...,...,...,...
31,2023-12,33.284219,OpenAQ
32,2024-01,31.507321,OpenAQ
33,2024-02,41.018535,OpenAQ
34,2024-03,86.125686,OpenAQ


In [69]:
import plotly.express as px


fig = px.line(df_pm25, x="year_month", y="pm25", color='source', title='PM2.5 (Mean) in Chiangmai')


fig.show()
fig.write_json(r'./time_series.json')
fig.write_html(r'./time_series.html')